In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import pandas as pd
import nbformat as nbf

import config_v2 as cfg
from library_report_v2 import Processing as pro

Baseline en config_v2.py: ['2023-01-01', '2024-12-23']
Study en config_v2.py: ['2024-12-23', '2025-01-13']


In [3]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='both')

In [4]:
#datos_csv = pd.DataFrame(df)
#datos_csv.to_csv(r'C:\Users\jpocampo\Desktop\Infome_Semanal_bc\CB_informes_Ubi\Informe_semanal_v2\data\datos_informe_3_semana_julio.csv', index=True)

In [5]:
set_devices_bl = set(df_bl['device_name'])
set_devices_st = set(df_st['device_name'])

set_devices = set_devices_bl.intersection(set_devices_st)

df_bl = None
df_st = None
df = None

blueprint_filepath = project_path / 'tools' / 'modelo.ipynb'
nb_blueprint = nbf.read(blueprint_filepath, as_version=4)

In [6]:
df_notebooks = pd.DataFrame(set_devices, columns=['device'])

In [7]:
df_notebooks

,device
0,BC 583 - Riosucio
1,BC 781 - Prado Plaza
2,BC 334 - El Peñol
3,BC 384 - Anapoima
4,BC 777 - Parque Washington
5,BC 415 - El Retiro
6,BC 332 - Zipaquira
7,BC 516 - Santa Marta
8,BC 388 - CC Hayuelos
9,BC 892 - La Vega


In [8]:
df_notebooks['code'] = df_notebooks['device'].str.split('-',expand=True)[0]

df_notebooks['number'] = df_notebooks['code'].str.strip('BC ')
df_notebooks['number'] = pd.to_numeric(df_notebooks['number'], errors='coerce')

df_notebooks = df_notebooks.sort_values(by='number')
sorted_devices = list(df_notebooks['device'])

In [9]:
for device_name in sorted_devices:
    nb = nb_blueprint.copy()
    nb_cells = nb['cells']
    cell_0 = nb_cells[0]
    cell_1 = nb_cells[1]
    cell_rest = nb_cells[2:]

    cell_0['source'] = f'# {device_name}'
    cell_1['source'] = f'DEVICE_NAME = \'{device_name}\'\nimport warnings\nwarnings.filterwarnings("ignore")'
    new_cells = [cell_0] + [cell_1] + cell_rest
    nb['cells'] = new_cells

    filename = f"Notebook {device_name}"
    write_path = project_path/'main'/'notebooks'/'individual'/f"{filename}.ipynb"
    nbf.write(nb, write_path)

    print(f"  - file: notebooks/individual/{filename}")


  - file: notebooks/individual/Notebook BC 73 - Pereira
  - file: notebooks/individual/Notebook BC 79 - La Quinta Ibague
  - file: notebooks/individual/Notebook BC 81 - Avenida Kennedy
  - file: notebooks/individual/Notebook BC 111 - Corozal
  - file: notebooks/individual/Notebook BC 115 - Circunvalar Pereira
  - file: notebooks/individual/Notebook BC 138 - Mosquera
  - file: notebooks/individual/Notebook BC 221 - Soacha
  - file: notebooks/individual/Notebook BC 265 - Valle de Lili
  - file: notebooks/individual/Notebook BC 332 - Zipaquira
  - file: notebooks/individual/Notebook BC 334 - El Peñol
  - file: notebooks/individual/Notebook BC 367 - Granada Meta
  - file: notebooks/individual/Notebook BC 384 - Anapoima
  - file: notebooks/individual/Notebook BC 385 - Villeta
  - file: notebooks/individual/Notebook BC 388 - CC Hayuelos
  - file: notebooks/individual/Notebook BC 415 - El Retiro
  - file: notebooks/individual/Notebook BC 461 - La carolina Unicentro
  - file: notebooks/individ